# **Пайплайны. Практика**

Импорт библиотек и данных

In [17]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV

from sklearn.datasets import fetch_california_housing
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.compose import make_column_transformer, make_column_selector
import joblib

In [5]:
df_wine= pd.read_csv('data/Red.csv')
df_wine.head()

df_wine_test= pd.read_csv('data/Red_test.csv')
df_wine_test.head()

,Name,Country,Region,Winery,Rating,NumberOfRatings,Price,Year
0,Lirac 2017,France,Lirac,Château Mont-Redon,4.3,100,15.50,2017
1,Erta e China Rosso di Toscana 2015,Italy,Toscana,Renzo Masi,3.9,100,7.45,2015
2,Bardolino 2019,Italy,Bardolino,Cavalchina,3.5,100,8.72,2019
3,Ried Scheibner Pinot Noir 2016,Austria,Carnuntum,Markowitsch,3.9,100,29.15,2016
4,Capatosta 2015,Italy,Toscana,Poggio Argentiera,3.8,101,19.90,2015


### **Задание 6.1**

Вам следует выполнить следующее:

* Добавить обработку столбца **'Region'** в пайплайн, полученный ранее в модуле, с использованием **OrdinalEncoder**.
  Важно! Для совпадения результатов процесс трансформации столбцов должен выполняться в следующей последовательности:
  1. Кодирование столбца 'Region'.
  2. Стандартизация столбца 'Price'.
  3. Кодирование столбца 'Country'.


* Обучить на тренировочном наборе данных пайплайн и оценить качество модели по метрике **RMSE** на тестовом наборе (файл Red_test.csv ).
* Зафиксировать random_state=42.
* Сохранить пайплайн в файл pipeline_wine.pkl.

In [12]:
def rmse(y_hat, y):
    return mean_squared_error(y_hat, y, squared = False)
random_state = 42

ct = make_column_transformer(
    (OrdinalEncoder(), ['Region']),
    (StandardScaler(), ['Price']),
    (OneHotEncoder(), ['Country']))

pipeline = Pipeline([('ct', ct), ('rf', RandomForestRegressor(random_state=random_state))])
X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']
pipeline.fit(X, y)

X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

joblib.dump(pipeline, 'data/pipeline_wine.pkl')

Качество по RSME: 0.0765


['data/pipeline_wine.pkl']

### **Задание 6.2**

Теперь попробуем изменить параметры случайного леса в пайплайне, полученном в предыдущем задании.

Измените параметр **n_estimators** в случайном лесу со значения по умолчанию до 200 , используя метод **set_params**.

В качестве ответа на задание введите в поле ниже полученный результат по метрике RMSE, округленный до четвёртого знака после запятой.

In [15]:
pipeline.set_params(rf__n_estimators = 200)
print(pipeline[1].n_estimators)
pipeline.fit(X, y)
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),4)}')

200
Качество по RSME: 0.0761


### **Задание 6.3**

Теперь попробуем добавить стекинг в качестве модели в пайплайн.

Вам следует выполнить следующее:

* Собрать **StackingRegressor**:
  1. В качестве базовых моделей возьмите ридж-регрессию **RidgeCV()** и решающее дерево.
  2. В качестве метамодели возьмите случайный лес с настройками (количество базовых моделей 10).
  3. Все базовые модели стекинга модели должны быть с настройками по умолчанию (кроме **random_state**).
* Зафиксировать **random_state=42** (для всех моделей).
* Заменить в пайплайне задачи 6.1 случайный лес на **StackingRegressor**.
* Обучить модель на тренировочной выборке.

В качестве ответа на задание введите в поле ниже полученный результат по метрике **RMSE**, округлённый до второго знака после запятой.

In [21]:
#Создаем список кортежей вида: (наименование модели, модель)
estimators = [
    ('rcv', RidgeCV()),
    ('dtr',  DecisionTreeRegressor(random_state=random_state))
]

#Создаем объект класса стекинг
reg = StackingRegressor(
    estimators=estimators,
    final_estimator=RandomForestRegressor(n_estimators=10,
                                          random_state=42)
)

pipeline = Pipeline([('ct', ct), ('reg', reg)])
X = df_wine[['Country', 'Price', 'Region']]
y = df_wine['Rating']
pipeline.fit(X, y)

X_test = df_wine_test[['Country', 'Price', 'Region']]
y_test = df_wine_test['Rating']
y_pred = pipeline.predict(X_test)
print(f'Качество по RSME: {round(rmse(y_test, y_pred),2)}')

Качество по RSME: 0.18
